In [72]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pprint import pprint

In [2]:
root = 'https://www.muscleandstrength.com/workouts/'
categories = ['men', 'women', 'muscle-building', 'fat-loss', 'strength', 'abs', 'full-body', 'sports', 'bodyweight', 'beginner', 'home', 'celebrity', 'cardio', 'chest', 'back', 'biceps', 'shoulders', 'legs', 'triceps', 'other']

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

all_links = [] 
for category in categories:
    print(f'==== NEW CATEGORY: {category} ====')
    driver.get(f"{root}{category}")
    btn = driver.find_elements(By.XPATH, "//li[@class='cell small-3']")[2].find_element(By.CSS_SELECTOR, "*")
    driver.execute_script("arguments[0].click()", btn)
    time.sleep(2)

    last_page = len(driver.find_elements(By.XPATH, "//li[@class='pager-item']")) + 1
    
    for i in range(1, last_page + 1):
        print('==== NEW PAGE ====')
        link_a = driver.find_elements(By.XPATH, "//div[@class='node-title']/a")
        for link in link_a:
            all_links.append(link.get_attribute('href'))
        
        if i != last_page:
            next_btn = driver.find_element(By.XPATH, "//li[@class='pager-next']")
            driver.execute_script("arguments[0].click()", next_btn)
            time.sleep(2)
    

len(all_links)

==== NEW CATEGORY: men ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW CATEGORY: women ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW CATEGORY: muscle-building ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW CATEGORY: fat-loss ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW CATEGORY: strength ====
==== NEW PAGE ====
==== NEW CATEGORY: abs ====
==== NEW PAGE ====
==== NEW CATEGORY: full-body ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW CATEGORY: sports ====
==== NEW PAGE ====
==== NEW CATEGORY: bodyweight ====
==== NEW PAGE ====
==== NEW CATEGORY: beginner ====
==== NEW PAGE ====
==== NEW CATEGORY: home ====
==== NEW PAGE ====
==== NEW CATEGORY: celebrity ====
==== NEW PAGE ====
==== NEW PAGE ====
==== NEW CATEGORY: cardio ====
==== NEW PAGE ====
==== NEW CATEGORY: chest ====
==== NEW PAGE ====
==== NEW CATEGORY: back ====
==== NEW 

1333

In [3]:
all_list = list(dict.fromkeys(all_links))
len(all_list)

456

In [12]:
links_df = pd.DataFrame({
     "links": all_list   
})
links_df.to_csv('links.csv', index=False)

In [22]:
links = pd.read_csv('links.csv')

In [109]:
goals = []
levels = []
workout_days = []
times = []
equipments = []
genders = []
workouts = []

In [111]:
options = webdriver.ChromeOptions()
options.add_experimental_option(
    "prefs", {
        "profile.managed_default_content_settings.images": 2,
    }
)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

for link in links['links']:
    print(f"{link}")
    driver.get(f"{link}")
    time.sleep(2)
    WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='node-stats-block']/ul/li")))
    summary_lis = driver.find_elements(By.XPATH, "//div[@class='node-stats-block']/ul/li")
    # goal = re.findall(r"(?<=Main Goal).*", summary_lis[0].text)
    goal = ""
    level = ""
    days = ""
    time_w = ""
    equipment = ""
    gender = ""

    for i in summary_lis:
        if 'Main Goal' in i.text:
            goal = i.text.splitlines()[1]
        elif 'Training Level' in i.text:
            level = i.text.splitlines()[1]
        elif 'Days Per Week' in i.text:
            len_days = i.text.splitlines()
            if len(len_days) < 2:
                days = 3
            else:
                days = len_days[1]
        elif 'Time Per Workout' in i.text:
            time_w = i.text.splitlines()[1]
        elif 'Equipment Required' in i.text:
            equipment = i.text.splitlines()[1].split(", ")
        elif 'Target Gender' in i.text:
            gender = i.text.splitlines()[1]

    if gender == 'Male & Female':
        gender = ['Male', 'Female']
    else:
        gender = [gender]
    
    lower_time = re.findall(r"[\d]+", time_w)
    if lower_time == []:
        time_w = 60
    else:
        time_w = sum(list(map(int, lower_time))) / len(lower_time)
    
    workout = []
    workout_each_days = driver.find_elements(By.XPATH, "//table")
    
    for workout_day in workout_each_days:
        rows = workout_day.find_elements(By.XPATH, "./tbody/tr")[1:]
        curr = []
        work = 0
        off = False
        for row in rows:
            if 'Complex' in row.text:
                if curr != []:
                    workout.append(curr)
                    curr = []
            elif 'Off' in row.text:
                off = True
                break
            else:
                if work > 0:
                    work = work - 1
                    continue
                info = row.find_elements(By.XPATH, "./td")

                if len(info) < 3:
                    continue
            
                if 'work up' in info[0].text:
                    exercise_name = re.findall(r"[\w]+(?=:)", info[0].text)[0]
                    sets = 3
                    reps = 5
                    work = 6 

                    curr.append({"exercise": exercise_name, "sets": sets, "reps": reps})
                else:
                    if 'Sets' in info[1].text:
                        continue
                    exercise_name = info[0].text
                    if '.' in exercise_name:
                        exercise_name = re.findall(r"(?<=. )[\w\s]+", exercise_name)[0]
                    sets = re.findall(r"[\d]+", info[1].text)
                    if len(sets) == 0:
                        continue
                    sets = math.ceil(sum(list(map(int, sets))) / len(sets))
                    reps = re.findall(r"[\d]+", info[2].text)
                    if len(reps) == 0: 
                        reps = 15
                    else:
                        reps = math.ceil(sum(list(map(int, reps))) / len(reps))
                    
                    curr.append({"exercise": exercise_name, "sets": sets, "reps": reps})
        if not off:
            workout.append(curr)

    for i in gender:
        goals.append(goal)
        levels.append(level)
        workout_days.append(days)
        times.append(time_w)
        equipments.append(equipment)
        genders.append(i)
        workouts.append(workout)
pprint(workouts)

https://www.muscleandstrength.com/workouts/body-god-complete-bodyweight-muscle-building-plan
https://www.muscleandstrength.com/workouts/no-juice-bodybuilding-workout-routine
https://www.muscleandstrength.com/workouts/10-week-fat-torcher
https://www.muscleandstrength.com/workouts/10-week-mass-building-program.html
https://www.muscleandstrength.com/workouts/advanced-fat-loss-workout
https://www.muscleandstrength.com/workouts/12-week-fat-destroyer
https://www.muscleandstrength.com/workouts/12-week-off-season-classic-bodybuilding-workout
https://www.muscleandstrength.com/workouts/12-week-fat-annihilation-workout
https://www.muscleandstrength.com/workouts/3-day-full-body-kettlebell-workout
https://www.muscleandstrength.com/workouts/3-day-full-body-workout-for-busy-people
https://www.muscleandstrength.com/workouts/3-day-PPL-workout-for-beginners
https://www.muscleandstrength.com/workouts/4-day-workout-to-build-muscle
https://www.muscleandstrength.com/workouts/3-day-workout-routine-and-diet-f

In [114]:
cell_res = {
    'Goals': goals,
    'Level': levels,
    'Days': workout_days,
    'Time': times,
    'Equipments': equipments,
    'Gender': genders,
    'Workout': workouts,
}

cell_data = pd.DataFrame(cell_res)
cell_data.to_csv('data.csv', index=False)